In [1]:
import pandas as pd
import pickle

In [2]:
data = pd.read_csv("/home/akanksha/Akanksha/Assignments/securly_Classification/dataset/test.csv")

data.drop(columns=['id'], inplace=True)

toxic_words = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

data.shape

(153164, 1)

In [3]:
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

porter = PorterStemmer()

stop_words = stopwords.words('english')

stop_words.append("hi")
stop_words.append("u")

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z]+"


# Function to remove unwanted data and stop words. 
# Also it tokenizes the data to converts the words in it's stem form.
def preprocess(text):
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip().split()
    tokens = [porter.stem(token) for token in text if token not in stop_words]
    return " ".join(tokens)

In [4]:
%%time
data['text'] = data.comment_text.apply(lambda x: preprocess(x))

CPU times: user 1min 42s, sys: 171 ms, total: 1min 43s
Wall time: 1min 43s


In [5]:
vectorization = pickle.load(open('vectorizer.pickle', 'rb'))

top_phrases = vectorization['top_phrases']
vectorizer = vectorization['vectorizer']

In [6]:
vectorized_data = vectorizer.transform(data['text'])

In [7]:
models = dict()
for category in toxic_words:
    model = pickle.load(open('Logistic_Regression/' + category + '.pickle', 'rb'))
    classifier = model[category]
    models[category] = classifier

In [8]:
%%time

result_category = []
for i in range(len(data)):
    result = []    
    for category in toxic_words:
        temp2 = []
        classifier = models[category]
        pred = classifier.predict(vectorized_data[i])
        if 1 in pred:
            result.append(category)            
    result_category.append(result)

CPU times: user 5min 33s, sys: 509 ms, total: 5min 33s
Wall time: 5min 33s


In [9]:
data['Result_Categories'] = result_category

In [10]:
%%time

result = []
for i in range(len(data)):
    res = []
    if len(data['Result_Categories']) > 0:
        for category in data['Result_Categories'][i]:
            tokens = data['text'][i].split()
            for token in tokens:
                if token in top_phrases[category] and token not in res:            
                    res.append(token)
        result.append(res)
    else:
        res.append([" "])

CPU times: user 3.75 s, sys: 7.97 ms, total: 3.75 s
Wall time: 3.75 s


In [11]:
data['Top_Phrases'] = result

In [12]:
data.head()

,comment_text,text,Result_Categories,Top_Phrases
0,Yo bitch Ja Rule is more succesful then you'll...,yo bitch ja rule succes ever what hate sad mof...,"[toxic, obscene, insult]","[get, ass, shit, bitch]"
1,== From RfC == \n\n The title is fine as it is...,rfc titl fine imo,[],[]
2,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",sourc zaw ashton lapland,[],[]
3,":If you have a look back at the source, the in...",look back sourc inform updat correct form gues...,[],[]
4,I don't anonymously edit articles at all.,anonym edit articl,[],[]


In [13]:
data.to_csv("Result.csv")